In [1]:
import pickle
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

with open('fixed_ground_truth.pkl','rb') as infile:
    data = pickle.load(infile)

/home/gary/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-09-15 17:02:56.472818: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 17:02:56.492706: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 17:02:56.492738: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 17:02:56.506051: I tensorflow/core/platform/cpu_feature_gua

In [2]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)
es = Elasticsearch("http://localhost:9200")

In [17]:
vector_data = []
for i in data:
    d = {}
    d['title'] = i['title']
    d['text'] = i['text']
    d['timecode_text'] = i['timecode_text']
    d['description'] = i['description']
    d['id'] = i['id']
    d['text_vector'] = model.encode(i['title']+' '+i['text']+' '+i['description'])
    vector_data.append(d)




In [28]:
count = 0
for i in data:
    if i['id']:
        count += 1
print(count)

837


In [19]:
len(vector_data[0]['text_vector'])

384

In [20]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "text": {"type": "text"},
            "timecode_text": {"type": "text"},
            "description": {"type": "keyword"},
            "id": {"type": "keyword"},
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
        }
    }


index_name = "vector-search"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vector-search'})

In [21]:
for doc in vector_data:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [22]:
def knn_query(question):
    return  {
        "field": "text_vector",
        "query_vector": model.encode(question),
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5,
        
    }

In [10]:
question = "When did we talk about Mage"

In [30]:
def vector_search(question):
    res = es_client.search(index=index_name, knn=knn_query(question), source=["id"])
    return res["hits"]["hits"]

In [31]:
vector_search(question)

[{'_index': 'vector-search',
  '_id': 'e0TL95EB5LG_H-Mr0L4B',
  '_score': 0.29429528,
  '_source': {'id': '5c9135a7b0adea3271a398bf9c80fc91'}},
 {'_index': 'vector-search',
  '_id': 'fETL95EB5LG_H-Mr0L4F',
  '_score': 0.29429528,
  '_source': {'id': '5c9135a7b0adea3271a398bf9c80fc91'}},
 {'_index': 'vector-search',
  '_id': 'fUTL95EB5LG_H-Mr0L4J',
  '_score': 0.29429528,
  '_source': {'id': '5c9135a7b0adea3271a398bf9c80fc91'}},
 {'_index': 'vector-search',
  '_id': 'RkTL95EB5LG_H-Mry71B',
  '_score': 0.28033978,
  '_source': {'id': '3cba0dbf73297ec686a5b91511d47dbe'}},
 {'_index': 'vector-search',
  '_id': 'R0TL95EB5LG_H-Mry71F',
  '_score': 0.28033978,
  '_source': {'id': '3cba0dbf73297ec686a5b91511d47dbe'}}]

In [32]:
from tqdm.auto import tqdm
relevance_total = []
for q in tqdm(data):
    doc_id = q['id']
    results = vector_search(q['student_question'])
    relevance = [d["_source"]['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/837 [00:00<?, ?it/s]

In [37]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)
print(f"VECTOR SEARCH Hit Rate is: {hit_rate(relevance_total)}")

VECTOR SEARCH Hit Rate is: 0.4540023894862604


In [38]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)
print(f"VECTOR SEARCH  MRR is: {mrr(relevance_total)}")

VECTOR SEARCH  MRR is: 0.6554958183990423


In [36]:
hybrid_data = []
for i in data:
    d = {}
    d['title'] = i['title']
    d['text'] = i['text']
    d['timecode_text'] = i['timecode_text']
    d['description'] = i['description']
    d['id'] = i['id']
    d['title_vector'] = model.encode(i['title'])
    d['timecode_vector'] = model.encode(i['timecode_text'])
    d['text_vector'] = model.encode(i['text'])
    d['description_vector'] = model.encode(i['description'])
    hybrid_data.append(d)




In [40]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "text": {"type": "text"},
            "timecode_text": {"type": "text"},
            "description": {"type": "keyword"},
            "id": {"type": "keyword"},
            "title_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "timecode_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "description_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
        }
    }


index_name = "hybrid-search"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'hybrid-search'})

In [41]:
for doc in hybrid_data:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [59]:

def knn_query(question, vector):
    return  {
        "field": f"{vector}",
        "query_vector": model.encode(question),
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5,
        
    }

In [60]:
def keyword_query(question):
    return {
        "bool": {
            "must": {
                "multi_match": {
                    "query": f"{question}",
                    "fields": ["description^3", "text", "title"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            },
        }
    }

In [61]:
def multi_search(key_word, vector):
    response = es_client.search(
        index=index_name,
        query=keyword_query(key_word),
        knn=knn_query(key_word, vector),
        size=10
    )
    return response["hits"]["hits"]

In [63]:
for vector in ['title_vector','timecode_vector','text_vector','description_vector']:
    from tqdm.auto import tqdm
    relevance_total = []
    for q in tqdm(data):
        doc_id = q['id']
        results = multi_search(q['student_question'], vector)
        relevance = [d["_source"]['id'] == doc_id for d in results]
        relevance_total.append(relevance)
        cnt = 0
        for line in relevance_total:
            if True in line:
                cnt = cnt + 1

    
    print(f"Hybrid Hit rate for {vector} is: {cnt / len(relevance_total)}")

  0%|          | 0/837 [00:00<?, ?it/s]

Hybrid Hit rate for title_vector is: 0.5543608124253285


  0%|          | 0/837 [00:00<?, ?it/s]

Hybrid Hit rate for timecode_vector is: 0.5639187574671446


  0%|          | 0/837 [00:00<?, ?it/s]

Hybrid Hit rate for text_vector is: 0.5579450418160096


  0%|          | 0/837 [00:00<?, ?it/s]

Hybrid Hit rate for description_vector is: 0.5519713261648745


In [66]:
for vector in ['title_vector','timecode_vector','text_vector','description_vector']:
    from tqdm.auto import tqdm
    relevance_total = []
    for q in tqdm(data):
        doc_id = q['id']
        results = multi_search(q['student_question'], vector)
        relevance = [d["_source"]['id'] == doc_id for d in results]
        relevance_total.append(relevance)
        total_score = 0.0
        for line in relevance_total:
            for rank in range(len(line)):
                if line[rank] == True:
                    total_score = total_score + 1 / (rank + 1)


    print(f"MRR is {vector}: {total_score / len(relevance_total)}")

  0%|          | 0/837 [00:00<?, ?it/s]

MRR is title_vector: 0.7265218751777894


  0%|          | 0/837 [00:00<?, ?it/s]

MRR is timecode_vector: 0.7416993798714236


  0%|          | 0/837 [00:00<?, ?it/s]

MRR is text_vector: 0.737086818000797


  0%|          | 0/837 [00:00<?, ?it/s]

MRR is description_vector: 0.7232325387343309


In [45]:
# def hit_rate(relevance_total):
#     cnt = 0

#     for line in relevance_total:
#         if True in line:
#             cnt = cnt + 1

#     return cnt / len(relevance_total)
# print(f"Hybrid Hit rate is: {hit_rate(relevance_total)}")

Hit rate is: 0.5579450418160096


In [46]:
# def mrr(relevance_total):
#     total_score = 0.0

#     for line in relevance_total:
#         for rank in range(len(line)):
#             if line[rank] == True:
#                 total_score = total_score + 1 / (rank + 1)

#     return total_score / len(relevance_total)
# print(f"MRR is: {mrr(relevance_total)}")

MRR is: 0.737086818000797
